In [1]:
import tensorflow as tf
import numpy as np
import time

In [2]:
x = np.zeros([10,10,10])
obj = ['sphere', 'cylinder', 'cube']
X = np.asarray([None])
Y = np.asarray([None])


for i in range(1000):
    
    shape = np.random.choice([0,1,2])
    xs = np.copy(x)
    
    if shape == 0:
        for j in range(10000):
            px, py, pz = np.random.uniform(0, 10, 3)
            dist = np.sqrt((px-5)**2 + (py-5)**2 + (pz-5)**2)
            if dist <= 3:
                a = int(np.floor(px))
                b = int(np.floor(py))
                c = int(np.floor(pz))
                xs[a,b,c] += 1
    
    elif shape == 1:
        for j in range(1000):
            px, py, pz = np.random.uniform(0, 10, 3)
            dist = np.sqrt((px-5)**2 + (py-5)**2)
            if (dist <= 3) and (np.abs(pz-5) <= 3):
                a = int(np.floor(px))
                b = int(np.floor(py))
                c = int(np.floor(pz))
                xs[a,b,c] += 1
                
    elif shape == 2:
        for j in range(1000):
            px, py, pz = np.random.uniform(0, 10, 3)
            if np.abs(px-5) <= 3 and np.abs(py-5) <= 3 and np.abs(pz-5) <= 3:
                a = int(np.floor(px))
                b = int(np.floor(py))
                c = int(np.floor(pz))
                xs[a,b,c] += 1
               
    if X.all() == None:
        X = np.expand_dims(xs, 0)
        Y = np.asarray([shape])
    else:
        X = np.concatenate([X, np.expand_dims(xs, 0)])
        Y = np.concatenate([Y, [shape]])
        
    del xs
del x

In [13]:
W1 = tf.Variable(tf.random.normal([5,5,5,1,8], 0, 0.1))
b1 = tf.Variable(tf.random.normal([8]))

W2 = tf.Variable(tf.random.normal([3,3,3,8,16],0,0.1))
b2 = tf.Variable(tf.random.normal([16]))

W3 = tf.Variable(tf.random.normal([3*3*3*16, 3],0,0.1))
b3 = tf.Variable(tf.random.normal([3]))

In [5]:
data = normalize(X)
data = np.expand_dims(data, axis=-1)
data = data.astype(np.float32)
labels = tf.one_hot(Y, 3, on_value=1., off_value=0., axis=-1)

In [4]:
def normalize(x):
    num, d, w, h = x.shape
    data = x.reshape(num, -1)
    size = data.shape[1]
    means = np.mean(data, axis=1)
    meansT = means.reshape(len(means), 1)
    stds = np.std(data, axis=1)
    stdsT = stds.reshape(len(stds), 1)
    normalized = (data-meansT)/stdsT
    normalized = normalized.reshape(num, d, w, h)
    return normalized

In [6]:
def conv_layer(x, W, b):
    conv = tf.nn.conv3d(x, W, strides=[1,2,2,2,1], padding='SAME')
    conv = tf.math.add(conv, b)
    return tf.nn.relu(conv)

In [7]:
def dense(x, W, b):
    return tf.matmul(x,W)+b

In [8]:
def model(x, W1, b1, W2, b2, W3, b3):
    num = x.shape[0]
    a1 = conv_layer(x, W1, b1)
    a2 = conv_layer(a1, W2, b2)
    flat = tf.reshape(a2, [num, -1])
    out = dense(flat, W3, b3)
    return out

In [70]:
def loss_func(corr, pred):
    u = -corr*tf.math.log(pred)-(1.-corr)*tf.math.log(1.-pred)
    a = tf.reduce_sum(u, axis = 1)
    b = tf.reduce_mean(a)
    return b

In [14]:
num_samples = len(data)
num_labels = len(labels)
batch = 64
epochs = 20
learning_rate = 0.01

In [15]:
for i in range(epochs):
    print("EPOCH", i)
    start = time.time()
    total_loss = 0.
    for j in range(0, num_samples, batch):
        x = data[j:j+batch]
        y = labels[j:j+batch]
        with tf.GradientTape(persistent=True) as t:
            out = model(x, W1, b1, W2, b2, W3, b3)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y))
            total_loss += loss
        W1.assign_sub(learning_rate*t.gradient(loss,W1))
        b1.assign_sub(learning_rate*t.gradient(loss,b1))
        W2.assign_sub(learning_rate*t.gradient(loss,W2))
        b2.assign_sub(learning_rate*t.gradient(loss,b2))
        W3.assign_sub(learning_rate*t.gradient(loss,W3))
        b3.assign_sub(learning_rate*t.gradient(loss,b3))
        del t
    end = time.time()
    mean_loss = total_loss/(num_samples//batch)
    model_op = model(data, W1, b1, W2, b2, W3, b3)
    correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    print("accuracy: ", accuracy.numpy())
    print("loss at end of EPOCH: %0.4f" % mean_loss)
    print("time taken: {:0.2f}s".format(end-start))
    print("Done with EPOCH")

EPOCH 0
accuracy:  0.676
loss at end of EPOCH: 1.0083
time taken: 0.44s
Done with EPOCH
EPOCH 1
accuracy:  0.718
loss at end of EPOCH: 0.5370
time taken: 0.42s
Done with EPOCH
EPOCH 2
accuracy:  0.799
loss at end of EPOCH: 0.3693
time taken: 0.42s
Done with EPOCH
EPOCH 3
accuracy:  0.888
loss at end of EPOCH: 0.2730
time taken: 0.53s
Done with EPOCH
EPOCH 4
accuracy:  0.931
loss at end of EPOCH: 0.2117
time taken: 0.53s
Done with EPOCH
EPOCH 5
accuracy:  0.955
loss at end of EPOCH: 0.1702
time taken: 0.51s
Done with EPOCH
EPOCH 6
accuracy:  0.97
loss at end of EPOCH: 0.1408
time taken: 0.42s
Done with EPOCH
EPOCH 7
accuracy:  0.984
loss at end of EPOCH: 0.1192
time taken: 0.42s
Done with EPOCH
EPOCH 8
accuracy:  0.989
loss at end of EPOCH: 0.1027
time taken: 0.42s
Done with EPOCH
EPOCH 9
accuracy:  0.993
loss at end of EPOCH: 0.0897
time taken: 0.51s
Done with EPOCH
EPOCH 10
accuracy:  0.996
loss at end of EPOCH: 0.0793
time taken: 0.51s
Done with EPOCH
EPOCH 11
accuracy:  0.996
loss a